# Energy Price Prediction - Neural Network Model Comparison

This notebook analyzes the performance of different neural network architectures for predicting real-time energy prices using Jiangsu Province electricity market data.

## Models Tested:
- **LSTM**: Long Short-Term Memory for sequential patterns
- **CNN**: Convolutional Neural Network for temporal feature extraction
- **GRU**: Gated Recurrent Unit (simpler alternative to LSTM)
- **Transformer**: Self-attention mechanism for complex dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

# Load results
results = pd.read_csv('model_comparison_summary.csv')
print("=== NEURAL NETWORK MODEL COMPARISON RESULTS ===")
print(results.round(3))

## Key Performance Metrics

- **MAE** (Mean Absolute Error): Average prediction error in RMB/MWh
- **RMSE** (Root Mean Square Error): Penalizes larger errors more heavily
- **R²** (R-squared): Explained variance (higher is better, max 1.0)
- **MAPE** (Mean Absolute Percentage Error): Error as percentage of actual values
- **Training Time**: Time to train the model in seconds
- **Epochs**: Number of training iterations before early stopping

In [ ]:
# Find best models for each metric
print("\n=== BEST MODELS BY METRIC ===")
print(f"🏆 Best MAE: {results.loc[results['MAE'].idxmin(), 'Model']} ({results['MAE'].min():.2f} RMB/MWh)")
print(f"🏆 Best RMSE: {results.loc[results['RMSE'].idxmin(), 'Model']} ({results['RMSE'].min():.2f} RMB/MWh)")
print(f"🏆 Best R²: {results.loc[results['R²'].idxmax(), 'Model']} ({results['R²'].max():.3f})")
print(f"🏆 Best MAPE: {results.loc[results['MAPE'].idxmin(), 'Model']} ({results['MAPE'].min():.2f}%)")
print(f"⚡ Fastest Training: {results.loc[results['Training_Time'].idxmin(), 'Model']} ({results['Training_Time'].min():.2f}s)")

# Overall ranking
print("\n=== OVERALL RANKING ===")
results_ranked = results.copy()
# Rank by RMSE (most important for price prediction)
results_ranked = results_ranked.sort_values('RMSE')
for i, row in results_ranked.iterrows():
    print(f"{row.name + 1}. {row['Model']} - RMSE: {row['RMSE']:.2f}, R²: {row['R²']:.3f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

metrics = ['MAE', 'RMSE', 'R²', 'MAPE', 'Training_Time']
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink']

for i, (metric, color) in enumerate(zip(metrics, colors)):
    bars = axes[i].bar(results['Model'], results[metric], color=color, alpha=0.7)
    axes[i].set_title(f'{metric} Comparison', fontsize=12, fontweight='bold')
    axes[i].set_ylabel(metric)
    axes[i].tick_params(axis='x', rotation=45)
    
    # Highlight best model
    if metric == 'R²':
        best_idx = results[metric].idxmax()
    else:
        best_idx = results[metric].idxmin()
    
    bars[best_idx].set_color('gold')
    bars[best_idx].set_edgecolor('red')
    bars[best_idx].set_linewidth(2)

# Model complexity comparison
axes[5].bar(results['Model'], results['Epochs'], color='purple', alpha=0.7)
axes[5].set_title('Training Epochs (Convergence Speed)', fontsize=12, fontweight='bold')
axes[5].set_ylabel('Epochs')
axes[5].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Model Performance Analysis

### 🥇 **Winner: CNN (Convolutional Neural Network)**
- **Best RMSE**: 84.64 RMB/MWh 
- **Best R²**: 0.414 (explains 41.4% of price variance)
- **Fastest Training**: 0.74 seconds
- **Good Convergence**: Only 8 epochs needed

### 🥈 **Runner-up: Transformer**
- **Best MAE**: 55.35 RMB/MWh
- **Best MAPE**: 20.89%
- **Good R²**: 0.396
- **Efficient**: Converged in 9 epochs

### Key Insights:
1. **CNN performs best overall** - Excellent at capturing temporal patterns in 15-minute price data
2. **Transformer excels at precision** - Lowest absolute errors (MAE, MAPE)
3. **All models achieve reasonable accuracy** - R² between 0.33-0.41
4. **Fast convergence** - All models converged within 10 epochs (early stopping)
5. **Price prediction is challenging** - ~85 RMB/MWh RMSE on prices averaging ~322 RMB/MWh

In [ ]:
# Display training visualizations
print("=== TRAINING HISTORY AND PREDICTIONS ===")
display(Image('model_comparison_results.png'))

In [ ]:
# Business impact analysis
print("=== BUSINESS IMPACT ANALYSIS ===")
print("\nBased on the EDA findings:")
print("- Average real-time price: 322.49 RMB/MWh")
print("- Price volatility (σ): 159.71 RMB/MWh")
print("- Best model RMSE: 84.64 RMB/MWh")
print("\nModel Performance Context:")
rmse_ratio = 84.64 / 322.49
volatility_ratio = 84.64 / 159.71
print(f"- RMSE as % of average price: {rmse_ratio*100:.1f}%")
print(f"- RMSE as % of natural volatility: {volatility_ratio*100:.1f}%")

print("\n💡 BUSINESS RECOMMENDATIONS:")
print("1. 🎯 Use CNN model for production - Best overall performance")
print("2. 📊 Consider ensemble of CNN + Transformer for critical decisions")
print("3. ⚡ CNN is fastest to retrain - Good for real-time applications")
print("4. 📈 Model explains 41% of price variance - Still need risk management")
print("5. 🔄 Retrain models frequently due to market dynamics")

In [ ]:
# Next steps and improvements
print("=== NEXT STEPS FOR MODEL IMPROVEMENT ===")
print("""\n🚀 IMMEDIATE IMPROVEMENTS:
1. Feature Engineering:
   - Add more lag features (24h, 48h)
   - Include weather data (temperature, wind speed)
   - Create interaction terms between renewables and load

2. Model Architecture:
   - Try ensemble methods (combining CNN + Transformer)
   - Experiment with different sequence lengths (6h, 12h, 24h)
   - Add attention mechanisms to CNN

3. Training Optimization:
   - Implement custom loss functions (MAPE-based)
   - Try different optimization strategies
   - Add regularization techniques

🎯 PRODUCTION DEPLOYMENT:
1. Real-time data pipeline integration
2. Model monitoring and drift detection
3. A/B testing framework for model comparison
4. Uncertainty quantification for risk management
5. Integration with existing arbitrage simulator
""")